In [8]:
import numpy as np
from typing import Union

In [9]:
class SVM:
    def __init__(self, C: float = 1.0, kernel: str = 'linear', degree: int = 3, gamma: Union[str, float] = 'auto',
                 coef0: float = 0.0, tol: float = 0.001, max_iter: int = 100) -> None:
        """
        Initializes a support vector machine instance.

        Args:
            C: Penalty parameter C of the error term. (default=1.0)
            kernel: Specifies the kernel type to be used. (default='linear')
            degree: Degree of the polynomial kernel function. Used only with polynomial kernel. (default=3)
            gamma: Kernel coefficient for 'rbf', 'poly' and 'sigmoid'. If gamma='auto' then gamma=1/n_features. (default='auto')
            coef0: Independent term in kernel function. It is only significant in 'polynomial' and 'sigmoid' kernels. (default=0.0)
            tol: Tolerance for stopping criterion
            max_iter: Maximum number of iterations for optimization algorithm
        """
        
        self.C = C
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.tol = tol
        self.max_iter = max_iter
        
    def _select_second_instance(self, first_instance_index: int, errors: np.ndarray) -> int:
        """
        Selects the second instance for the SMO algorithm.

        Args:
            first_instance_index: Index of the first instance.
            errors: Array containing the errors of each instance.

        Returns:
            Index of the second instance.
        """
        
        # Random choice from erros
        second_instance_index = np.random.choice(errors.shape[0])
        
        # While are the same choose a new value randomly
        while second_instance_index == first_instance_index:
            second_instance_index = np.random.choice(errors.shape[0])
            
        return second_instance_index
    
    def _compute_error(self, instance_index: int) -> float:
        """
        Computes the error of a given istance index.

        Args:
            instance_index: Index of the instance.

        Returns:
            The error of the instance.
        """
        
        return self.predict(self.X[instance_index]) - self.y[instance_index]
    
    def _compute_kernel_matrix(X: np.ndarray) -> np.ndarray:
        """
        Computes the kernel matrix given the kernel defined as part of the constructor
        
        Args:
            X: A numpy array containing the data
            
        Returns:
            Numpy array transformed
            
        Raises:
            ValueError: If the specified kernel type is not supported
        """
        
        # When gamma is auto then 1/features
        if self.gamma == 'auto':
            self.gamma = 1 / n_features

        # Linear kernel
        if self.kernel == 'linear':
            K = np.dot(X, X.T)
        # Polynomial kernel
        elif self.kernel == 'poly':
            K = (self.gamma * np.dot(X, X.T) + self.coef0) ** self.degree
        # Radial Basis Function kernel
        elif self.kernel == 'rbf':
            K = np.exp(-self.gamma * ((X[:, np.newaxis] - X) ** 2).sum(axis=2))
        # Sigmoid kernel
        elif self.kernel == 'sigmoid':
            K = np.tanh(self.gamma * np.dot(X, X.T) + self.coef0)
        else:
            raise ValueError('Invalid kernel. The given kernel is not supported')
        return K
    
    def _compute_kernel_between_samples(self, x_i: np.ndarray, x_j: np.ndarray) -> float:
        """
        Calculates the kernel function value between two input vectors.

        Args:
            x_i: An input vector of shape (n_features,)
            x_j: An input vector of shape (n_features,)

        Returns:
            The value from the kernel function between the input vectors

        Raises:
            ValueError: If the specified kernel type is not supported
        """
        
        # When gamma is auto then 1/features
        if self.gamma == 'auto':
            self.gamma = 1 / n_features
            
        # Linear kernel
        if self.kernel == 'linear':
            kv = np.dot(x_i, x_j)
        # Polynomial kernel
        elif self.kernel == 'poly':
            kv = (self.gamma * np.dot(x_i, x_j) + self.coef0) ** self.degree
        # Radial Basis Function kernel
        elif self.kernel == 'rbf':
            kv = np.exp(-self.gamma * np.linalg.norm(x_i - x_j) ** 2)
        # Sigmoid kernel
        elif self.kernel == 'sigmoid':
            kv = np.tanh(self.gamma * np.dot(x_i, x_j) + self.coef0)
        else:
            raise ValueError('Invalid kernel. The given kernel is not supported')        
        return kv
        
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Fits the SVM model on the given training data. 
        For this implementation the "Pseudo-Code for Simplified SMO" is used as based. 
        You can find it here: http://cs229.stanford.edu/materials/smo.pdf 

        Args:
            X: A numpy array of shape (m, n) with training data.
            y: A numpy array of shape (m, ) with the target values for the training data.
        """
        
        # Get the number of samples and features
        n_samples, n_features = X.shape
        
        # Init the alpha as an array of zeros with the same 
        #   lenght as n_samples and the bias term as zero
        self.alpha = np.zeros(n_samples)
        self.b = 0
        
        # Comput the kernel converted version of X
        kernel_X = self._compute_kernel_matrix(X)
        
        # Init passes
        passes = 0
        # While the number of iteration is less than the max iterations allowed 
        while passes < self.max_iter:
            # Init the number of alphas that are changed
            # Iterate over samples in X
            for i in range(n_samples):
                #  Calculate E_i (the error for sample i)
                E_i = self._compute_error(i)
                
                # If the current alpha_i fails to the Karush-Kuhn-Tucker (KKT) conditions, 
                #   then look for a second instance j to optimize
                if (y[i]*E_i < -self.tol and self.alpha[i] < self.C) or (y[i]*E_i > self.tol and self.alpha[i] > 0):
                    # Get the second instance, considering to select j <> i randomly
                    j = self._select_second_instance(i, E_i)
                    #  Calculate E_k (the error for sample j)
                    E_j = self._compute_error(j)
                    
                    # Get the old values of alpha i and j
                    alpha_i_old, alpha_j_old = self.alpha[i], self.alpha[j]
                    
                    # Calculates the bounds for the new alpha_j based on the equality of y_i and y_j
                    # When not equal
                    if y[i] != y[j]:
                        L = max(0, self.alpha[j] - self.alpha[i])
                        H = min(self.C, self.C + self.alpha[j] - self.alpha[i])
                    # When equals
                    else:
                        L = max(0, self.alpha[i] + self.alpha[j] - self.C)
                        H = min(self.C, self.alpha[i] + self.alpha[j])
                    # Skip the iteration if L and H (the bounds) are equals
                    if L == H:
                        continue
                        
                    # Computes η (eta)
                    eta = 2 * kernel_X[i, j] - kernel_X[i, i] - kernel_X[j, j]
                    # Skip the iteration if eta is negative
                    if eta >= 0:
                        continue
                        
                    # Compute the new value for alpha_j, cutting it to the bounds
                    self.alpha[j] -= (y[j] * (E_i - E_j)) / eta
                    # Cutting to the bounds
                    #   In the original SMO document there is a function by parts
                    #   But here we use a compare with min to ensure that alpha_j
                    #   is less than H (upper bound). Then we use, a max with L (lower bound)
                    #   to ensure that alpha_j will be greater than the latter value
                    self.alpha[j] = min(H, self.alpha[j])
                    self.alpha[j] = max(L, self.alpha[j])
                    
                    # Skip the iteration if the absolute difference between 
                    #   the old and new values of alpha_j is less than a tolerance
                    if abs(self.alpha[j] - alpha_j_old) < self.tol:
                        continue
                        
                    # Compute the new alpha_i with the formula specified in the SMO algorithm
                    self.alpha[i] += y[i] * y[j] * (alpha_j_old - self.alpha[j])
                    
                    # Calculate the new value for b
                    b1 = self.b - E_i - y[i] * (self.alpha[i] - alpha_i_old) * kernel_matrix[i, i] - \
                        y[j] * (self.alpha[j] - alpha_j_old) * kernel_matrix[i, j]
                    b2 = self.b - E_j - y[i] * (self.alpha[i] - alpha_i_old) * kernel_matrix[i, j] - \
                        y[j] * (self.alpha[j] - alpha_j_old) * kernel_matrix[j, j]
                    
                    # Calculate the value for b 
                    self.b = b1 if 0 < self.alpha[i] < self.C else b2 if 0 < self.alpha[j] < self.C else (b1 + b2) / 2
                    # Increment by one the alphas changed
                    count_changed_alphas += 1
                    
            # No changes made during the current iteration -> increment
            if count_changed_alphas == 0:
                passes += 1
            # Set to zero otherwise
            else:
                passes = 0
        
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Computes the prediction of a given set
        
        Args:
            X: Data to make predictions from
        Returns:
            Numpy array with the label classes
        """
        
        # Array to save the predicted labels
        y_pred = np.zeros(X.shape[0])

        # Get indexes of the support vectors, getting those with non-zero alphas
        support_vector_indices = np.where(self.alpha != 0)[0]

        # Get SV, labels and alphas using the indexes from before
        support_vectors = self.X[support_vector_indices]
        support_vector_labels = self.y[support_vector_indices]
        support_vector_alphas = self.alpha[support_vector_indices]

        # Iterate over each example in the given set
        for i, x in enumerate(X):
            # Get the kernelized values between x and SV
            kernel_values = self._compute_kernel_between_samples(x, support_vectors)

            # Calculate the decision function value for the current x
            decision_function = np.sum(support_vector_labels * support_vector_alphas * kernel_values) + self.b

            # Use sign function to calculate the predicted label
            y_pred[i] = np.sign(decision_function)

        # Return the predicted labels
        return y_pred

In [ ]:
# TODO: Test the implementation